In [9]:
from nl2sql.database.mysql_connector import MySQLConnector
from core.vector_database.faiss_wrapper import FaissWrapper
db_user = "root"
db_password = ""
db_host = "localhost"
db_name = "classicmodels"


db = MySQLConnector.from_uri_db(
    host=db_host,
    user=db_user,
    password=db_password,
    db_name=db_name,
)
db_summary = db.get_db_summary()
faiss_wrapper = FaissWrapper(text_chunks=db_summary)
res = faiss_wrapper.search_for_chunks("give me details of customer and their order count", top_k=3)
print(res)

['orders(orderNumber, orderDate, requiredDate, shippedDate, status, comments, customerNumber), and index keys: customerNumber(`customerNumber`) ', 'orderdetails(orderNumber, productCode, quantityOrdered, priceEach, orderLineNumber), and index keys: productCode(`productCode`) ', 'customers(customerNumber, customerName, contactLastName, contactFirstName, phone, addressLine1, addressLine2, city, state, postalCode, country, salesRepEmployeeNumber, creditLimit), and index keys: salesRepEmployeeNumber(`salesRepEmployeeNumber`) ']


#### GET SQL Query

In [10]:
from constants.type import GenerateSQLResponse
from constants.prompts import GENERATE_SQL_PROMPTS
from model.llm import DMetaLLM

client = DMetaLLM()
query = "what is price of `1968 Ford Mustang`"
res = client.get_structured_response(
    GENERATE_SQL_PROMPTS.format(table_info=db_summary, input=query),
    response_format=GenerateSQLResponse,
)
print(res["sql_query"])

SELECT `productCode`, `productName`, `buyPrice` FROM `products` WHERE `productName` = '1968 Ford Mustang' LIMIT 5


#### Execute SQL Query

In [3]:
sql_res =  db.run_no_throw(res["sql_query"])
print(sql_res)

[('S12_1099', '1968 Ford Mustang', Decimal('95.34'))]


#### Rephrasing SQL query answer

In [6]:
from constants.prompts import SQL_QUERY_ANSWER_PROMPTS

res = client.get_response(query=SQL_QUERY_ANSWER_PROMPTS.format(question=query, query=res["sql_query"], result=sql_res))
print(res)

The price of the `1968 Ford Mustang` is $95.34.
